In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.optim import Adam

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
train_dataset = torchvision.datasets.MNIST(root='G:/py-py-py-pytorch/gnn/data',train=True,download=True,
                           transform=transforms.Compose([transforms.ToTensor()]))
train_dataloader = DataLoader(train_dataset,batch_size = 16,shuffle=True)

100%|██████████| 9912422/9912422 [00:03<00:00, 3202014.94it/s]


Extracting G:/py-py-py-pytorch/gnn/data\MNIST\raw\train-images-idx3-ubyte.gz to G:/py-py-py-pytorch/gnn/data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 8449755.43it/s]


Extracting G:/py-py-py-pytorch/gnn/data\MNIST\raw\train-labels-idx1-ubyte.gz to G:/py-py-py-pytorch/gnn/data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 3667573.18it/s]


Extracting G:/py-py-py-pytorch/gnn/data\MNIST\raw\t10k-images-idx3-ubyte.gz to G:/py-py-py-pytorch/gnn/data\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 1336504.05it/s]

Extracting G:/py-py-py-pytorch/gnn/data\MNIST\raw\t10k-labels-idx1-ubyte.gz to G:/py-py-py-pytorch/gnn/data\MNIST\raw



In [34]:
test_dataset = torchvision.datasets.MNIST(root='G:/py-py-py-pytorch/gnn/data',train=False,download=True,
                           transform=transforms.Compose([transforms.ToTensor()]))
test_dataloader = DataLoader(test_dataset,batch_size = 16,shuffle=True)

In [8]:
it = iter(train_dataloader)
next(it)[0].shape

torch.Size([16, 1, 28, 28])

In [19]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        # [16,1,28,28] => [16,16,26,26]
        self.conv_1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3) #
        # after flatten, size is [16,16*26*26]
        self.lin_1 = nn.Linear(16*26*26,128)
        self.lin_2 = nn.Linear(128,10)

    def forward(self,x):
        # [16,1,28,28] => [16,16,26,26]
        x = self.conv_1(x)
        x = F.relu(x)
        #16x(16x26x26)
        x = x.flatten(start_dim=1)

        x = self.lin_1(x)
        x = F.relu(x)

        logits = self.lin_2(x)
        out = F.softmax(logits,dim=1)
        return out

In [15]:
# t = torch.rand(32,32,26,26)
# t = t.flatten(start_dim=1)
# t.shape

torch.Size([32, 21632])

In [36]:
# Params
learning_rate = 3e-4
num_epochs = 5
model = NeuralNetwork()
model = model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=learning_rate)

In [38]:
# training
for epoch in range(num_epochs):
    train_acc = 0.0
    train_loss = 0.0

    for i, (img,label) in enumerate(train_dataloader):
        imgs = img.to(device)
        labels = label.to(device)

        #forward
        preds = model(imgs)
        #loss
        loss = loss_function(preds,labels)
        #backprop
        optimizer.zero_grad()
        loss.backward()

        #update wts
        optimizer.step()

        train_loss += loss.detach().item()
        train_acc += (torch.argmax(preds,1).flatten() == labels).type(torch.float32).mean().item()

    print("Epoch: %d || Loss: %.2f || Train Acc: %.2f" %(epoch,train_loss/i ,train_acc/i))


Epoch: 0 || Loss: 1.65 || Train Acc: 0.82
Epoch: 1 || Loss: 1.52 || Train Acc: 0.95
Epoch: 2 || Loss: 1.50 || Train Acc: 0.96
Epoch: 3 || Loss: 1.49 || Train Acc: 0.97
Epoch: 4 || Loss: 1.48 || Train Acc: 0.98


In [63]:
#evaluation
with torch.no_grad():
    test_acc = 0.0
    for i, (imgs,labels) in enumerate(test_dataloader,1):
        imgs = imgs.to(device)
        labels = labels.to(device)
        outs = model(imgs)
        test_acc += (torch.argmax(outs,1).flatten() == labels).type(torch.float32).mean().item()
    
print(test_acc/i)

0.9761
